In [2]:
import pandas as pd
ds = pd.read_parquet("/home/users/hc387/data/sft_data/Qwen3b-instruct/correct/zebra_puzzles.parquet")
print(ds.iloc[0])

prompt        <|im_start|>system\nYou are a helpful assistan...
completion     \nFrom the information provided, here's what ...
Name: 0, dtype: object


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pickle

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B-Instruct")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-3B-Instruct")
model.to(device)

/home/users/hc387/miniconda3/envs/zero/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/users/hc387/miniconda3/envs/zero/lib/python3.9/site-packages/transformers/utils/hub.py:106: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.51s/it]


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-35): 36 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear(in_features=2048, out_features=256, bias=True)
          (v_proj): Linear(in_features=2048, out_features=256, bias=True)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (up_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((2048,), eps=1e-06)
    (rotary_emb):

In [4]:
class OnlineStats:
    """
    Maintains statistics using an online update strategy:
      - running_mean: Average value for each feature dimension
      - running_var:  Variance approximation for each feature dimension
      - running_l2:   Mean of squared values (L2) for each feature dimension

    The idea is that when receiving a new batch x, we scale the existing statistics and blend them with the new batch 
    in the reverse proportion. This allows us to dynamically estimate the mean, approximate variance, and L2 mean 
    without storing all the sample data.
    """

    def __init__(self, hidden_dim=None):
        self.hidden_dim = hidden_dim

        self.running_mean = None  # [D], running mean of features
        self.running_var  = None  # [D], running variance of features
        self.running_l2   = None  # [D], mean of squared features (L2)
        self.sample_count = 0     # Total number of samples (including batch and sequence dimensions)

    def update(self, x: torch.Tensor):
        """
        Update the statistics using a new batch x.
        Supports input shapes of (batch, length, dim) or (N, dim); internally flattened to (N, D).

        Steps:
          1. Flatten to [N, D]
          2. Clamp to avoid extreme values or overflows
          3. If it's the first update, directly initialize the statistics
          4. If statistics already exist, update them using online merging strategy
        """
        # 1. Flatten => [N, D]
        x = x.view(-1, x.shape[-1])

        # 2. Clamp to avoid extreme values causing floating point overflow
        x = x.to(torch.float32)  # Or x.float()

        # Debug: Check for Inf/NaN
        if torch.isinf(x).any():
            print(f"[OnlineStats] After clamp, x still has Inf, shape={x.shape}")
        if torch.isnan(x).any():
            print(f"[OnlineStats] x has NaN after clamp, shape={x.shape}")

        n_new = x.shape[0]
        
        # 3. If it's the first update, initialize the statistics
        if self.sample_count == 0:
            self.hidden_dim = x.shape[-1]
            self.running_mean = x.mean(dim=0)        # [D]
            self.running_var  = torch.zeros_like(self.running_mean)
            self.running_l2   = (x ** 2).mean(dim=0) # [D]
            self.sample_count = n_new
            return

        # 4. If statistics already exist, update using online merging
        old_mean = self.running_mean.clone()
        total_count = self.sample_count + n_new

        alpha_old = self.sample_count / total_count
        alpha_new = n_new / total_count

        batch_mean = x.mean(dim=0)  # [D]
        self.running_mean = alpha_old * self.running_mean + alpha_new * batch_mean

        # 4.1 Update variance (running_var)
        if total_count > 1:
            self.running_var *= (self.sample_count - 1) / (total_count - 1)

        diff_sum = ((x - self.running_mean) * (x - old_mean)).sum(dim=0)  # [D]

        self.running_var += diff_sum / total_count

        # 4.2 Update L2 mean
        batch_l2 = (x ** 2).mean(dim=0)  # [D]
        self.running_l2 = alpha_old * self.running_l2 + alpha_new * batch_l2

        # 4.3 Update sample count
        self.sample_count = total_count

    def get_stats(self):
        """
        Return the current statistics: mean, variance, and L2.
        """
        if self.sample_count == 0:
            return {
                "mean": None,
                "var":  None,
                "l2":   None
            }
        return {
            "mean": self.running_mean,
            "var":  self.running_var,
            "l2":   self.running_l2
        }

    def reset(self):
        """
        Reset the statistics and clear recorded data.
        """
        self.running_mean = None
        self.running_var  = None
        self.running_l2   = None
        self.sample_count = 0

In [8]:
layer_activations = {}

def init_stats_dict():
    return {
        'attention_input_states': OnlineStats(),
        'attention_post_aggregation': OnlineStats(),
        'mlp_input_states': OnlineStats(),
        'mlp_intermediate_states': OnlineStats()
    }

def q_proj_hook(module, input, output):
    layer_activations[layer_idx]['attention_input_states'].update(
        input[0].detach().cpu()
    )

def o_proj_hook(module, input, output):
    layer_activations[layer_idx]['attention_post_aggregation'].update(
        input[0].detach().cpu()
    )

def gate_proj_hook(module, input, output):
    layer_activations[layer_idx]['mlp_input_states'].update(
        input[0].detach().cpu()
    )

def down_proj_hook(module, input, output):
    layer_activations[layer_idx]['mlp_intermediate_states'].update(
        input[0].detach().cpu()
    )


for i, layer in enumerate(model.model.layers):
    layer_activations[i] = init_stats_dict()
    layer.self_attn.q_proj.register_forward_hook(q_proj_hook)
    layer.self_attn.o_proj.register_forward_hook(o_proj_hook)
    layer.mlp.gate_proj.register_forward_hook(gate_proj_hook)
    layer.mlp.down_proj.register_forward_hook(down_proj_hook)

    